In [2]:
import pandas as pd
import numpy as np

import os
import re

from difflib import SequenceMatcher

In [64]:
src_dir = 'txt'  # open exported files
bname = 'safe4'
with open(os.path.join(src_dir, bname + '.txt')) as fp:
    txt = fp.read()
rows = txt.split(';')

In [65]:
len(rows)  # ensure you have the number of questions

46

In [66]:
questions, options, answers = [],[],[]
qoptions = []

In [67]:
#  start splitting with set separators
for i, r in enumerate(rows):
    try:
        q, a = r.split('|')
        qoptions.append(q)
        answers.append(a)
    except Exception as e:
        print(i, e)
        print(r)

45 not enough values to unpack (expected 2, got 1)



In [68]:
for i, qo in enumerate(qoptions):
    try:
        q, o = re.split('[\?]+', qo)
        questions.append(q)
        options.append(o)
    except Exception as e:
        print("Could not split question, options from: ", i, e)  # option contains \?+ pattern
        print(qo)
        questions.append(qo)
        options.append("")

In [69]:
print(len(questions), len(answers), len(options))
assert len(questions) == len(answers) and len(answers) == len(options)

45 45 45


In [70]:
def print_answers(user_question, questions, answers, top=10):
    comparison_scores = [-SequenceMatcher(None, user_question, q).ratio() for q in questions]    
    idx_sort = np.argsort(comparison_scores)
    for i in idx_sort[:top]:
        print(i, questions[i], '\nR = ', answers[i], '\n')

In [71]:
user_question = 'A SAFe scrum master demonstrates which servant leadership behaviour'
print_answers(user_question, questions, answers, top=5)

22 what is an example of SAfe scrum master servant leader behavior 
R =  -uses persuasion instead of authority 

6 the scrum master role includes traditional scrum team leadership and responsibilities to which other group 
R =  -to the agile teams at the program level 

16 what is the key scrum master responsibility when estimating stories 
R =  -ensure everyone on the team participates 

23 the scrum master is what above all else 
R =  -a servant leader 

29 how can a scrum master support a problem solving workshop 
R =  -by providing facilitation to breakout groups focused on specific problems 



In [72]:
df = pd.DataFrame({'questions': questions, 'options': options, 'anwsers': answers})
df.to_csv(os.path.join('csv', bname + '.csv'), index=False)  # save info 

# Merging csv

In [83]:
fnames = os.listdir('csv')
df = pd.read_csv(os.path.join('csv', fnames[0]))
for fname in fnames[1:]:
    df_it = pd.read_csv(os.path.join('csv', fname))
    df = df.append(df_it)

In [84]:
df.columns

Index(['questions', 'options', 'anwsers'], dtype='object')

In [85]:
df['questions'] = df['questions'].apply(lambda c: c.lower().strip())  # to lowercase and trim questions

In [86]:
print(df.shape)
df.drop_duplicates(['questions'], inplace=True)
print(df.shape)

(129, 3)
(119, 3)


In [77]:
df.to_csv('merged.csv', index=False)

In [78]:
questions = df['questions'].values
options = df['options'].values
answers = df['anwsers'].values

In [79]:
questions[0]

'a scrum master notices that the team members are not open to showcasing their work. which core value would the scrum master encourage the team to adopt'

In [87]:
u_question = 'How does the Scrum Master provide the most value to the team'
print_answers(u_question, questions, answers)

61 how does the scrum master provide the most value to the team 
R =  By removing impediments for the team 

16 what does an effective scrum master help the team with 
R =  Relentless improvement 

108 what is the scrum master's role in the i&a 
R =  To facilitate one of the teams in the problem solving workshop 

39 why does a scrum master use the powerful questioning technique 
R =  To guide people to a solution 

55 an effective scrum master helps the team with what 
R =  Relentless improvement 

63 what is the scrum master's role in team breakout #1 
R =  Facilitate the coordination with other teams for dependencies 

127 what is the key scrum master responsibility when estimating stories 
R =  -ensure everyone on the team participates 

29 what is the primary purpose of the ip iteration 
R =  Provide opportunity to identify more effective ways of working 

43 the safe scrum master role includes responsibilities to which other group 
R =  The other Agile Teams on the Agile release 